# Import

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install opencv-python-headless
# !pip install openpyxl

import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import os
from PIL import Image, ImageDraw
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import os
import glob
from tqdm import tqdm

# Image Pre-processing  
**Image 폴더에 있는 캡쳐된 사진들을 편집하여 Edited 폴더에 저장합니다.**

In [ ]:
# 이미지 경로
dir_path = "./drive/MyDrive/Colab Notebooks/LPBA_3/Image/"
output_dir = "./drive/MyDrive/Colab Notebooks/LPBA_3/Edited/"

# 출력 디렉토리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)



# 특정 이미지만 사용하는 경우
img_name_list = [f'{str(i).zfill(4)}.png' for i in range(1, 2116)]  # 2421개의 이미지면 2422 입력
img_list = [dir_path + img_name for img_name in img_name_list]  # 경로 수정

# # dir_path에 있는 모든 PNG 파일을 사용하는 경우
# img_list = [os.path.join(dir_path, img_name) for img_name in os.listdir(dir_path) if img_name.endswith('.png')]



# 당구대만 뽑기 + 왜곡 보정하기 (특정 이미지만 사용하는 경우)
img_name_list = [f'{str(i).zfill(4)}.png' for i in range(1, 2116)]  # 2421번까지의 이미지면 2422 입력
img_list = [dir_path + img_name for img_name in img_name_list]  # 경로 수정

# # dir_path에 있는 모든 PNG 파일을 사용하는 경우
# img_list = [os.path.join(dir_path, img_name) for img_name in os.listdir(dir_path) if img_name.endswith('.png')]



# 당구대만 뽑기 + 왜곡 보정하기
def correct_perspective(image_path):
    # 이미지 읽기
    img = cv2.imread(image_path)

    if img is None:
        print(f"Failed to load image: {image_path}")

        return None

    # 이미지를 RGB에서 HSV 색상 공간으로 변환
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Trial 1. 당구대 색상 범위 설정
    lower_color = np.array([90, 100, 100])  # 하한 값 (추정)
    upper_color = np.array([130, 255, 255])  # 상한 값 (추정)

    # 해당 색상 범위에 해당하는 부분을 마스크로 추출
    mask = cv2.inRange(hsv, lower_color, upper_color)

    # 중앙 부분만 강조하기 위해 ROI(Region of Interest) 마스크 추가
    h, w, _ = img.shape
    center_mask = np.zeros_like(mask)
    center_mask[int(h*0.10):int(h*0.90), int(w*0.10):int(w*0.90)] = 255
    mask = cv2.bitwise_and(mask, center_mask)

    # 마스크를 통해 당구대만 남긴 이미지를 생성
    masked_img = cv2.bitwise_and(img, img, mask=mask)

    # 그레이스케일 변환
    gray = cv2.cvtColor(masked_img, cv2.COLOR_BGR2GRAY)

    # 블러링을 통해 노이즈 제거
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 이미지에서 엣지 검출 (Canny 알고리즘)
    edges = cv2.Canny(blurred, 50, 150)

    # Hough 변환을 사용하여 직선 검출
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=10)

    # 모폴로지 연산으로 노이즈 제거
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # 외곽선을 찾음
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # 사각형에 가까운 컨투어 필터링
        valid_contours = []

        for contour in contours:
            area = cv2.contourArea(contour)

            if area > 500:  # 면적이 너무 작지 않은 것만 필터링
                epsilon = 0.02 * cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, epsilon, True)

                if len(approx) >= 4:  # 사각형 또는 다각형
                    valid_contours.append(approx)

        if valid_contours:
            # 가장 큰 컨투어 선택
            largest_contour = max(valid_contours, key=cv2.contourArea)

            # Convex Hull 적용
            hull = cv2.convexHull(largest_contour)

            # 다각형으로 근사화
            epsilon = 0.02 * cv2.arcLength(hull, True)
            approx = cv2.approxPolyDP(hull, epsilon, True)

            # 사각형(당구대)이 맞는지 확인
            if len(approx) >= 4:
                # 좌표를 순서대로 정렬 (왼쪽 상단, 오른쪽 상단, 오른쪽 하단, 왼쪽 하단)
                pts = approx.reshape(-1, 2)
                rect = np.zeros((4, 2), dtype="float32")

                s = pts.sum(axis=1)
                diff = np.diff(pts, axis=1)

                rect[0] = pts[np.argmin(s)]  # 왼쪽 상단
                rect[2] = pts[np.argmax(s)]  # 오른쪽 하단
                rect[1] = pts[np.argmin(diff)]  # 오른쪽 상단
                rect[3] = pts[np.argmax(diff)]  # 왼쪽 하단

                # 변환 후의 좌표 설정 (직사각형)
                (tl, tr, br, bl) = rect
                widthA = np.linalg.norm(br - bl)
                widthB = np.linalg.norm(tr - tl)
                maxWidth = max(int(widthA), int(widthB))

                heightA = np.linalg.norm(tr - br)
                heightB = np.linalg.norm(tl - bl)
                maxHeight = max(int(heightA), int(heightB))

                # 직사각형으로의 변환 좌표
                dst = np.array([
                    [0, 0],
                    [maxWidth - 1, 0],
                    [maxWidth - 1, maxHeight - 1],
                    [0, maxHeight - 1]], dtype="float32")

                # 투영 변환 행렬 계산
                M = cv2.getPerspectiveTransform(rect, dst)

                # 이미지 변환 적용
                warped = cv2.warpPerspective(img, M, (maxWidth, maxHeight))

                return warped

    print(f"Error: No valid contours found in {image_path}")

    return None



# 특정 이미지만 사용한 경우
num_images_to_show = 2115 # 2163번까지의 이미지 사용이면 2163 입력

for i, img_path in enumerate(img_list[:num_images_to_show]):
    corrected_img = correct_perspective(img_path)

    if corrected_img is not None:
        # OpenCV 이미지를 Matplotlib에서 사용하기 위해 BGR에서 RGB로 변환
        corrected_img_rgb = cv2.cvtColor(corrected_img, cv2.COLOR_BGR2RGB)

        # 수정된 이미지 저장
        output_path = os.path.join(output_dir, f'{str(i + 1).zfill(4)}.png')

        # # 원본 이미지 이름 그대로 저장하는 경우
        # output_path = os.path.join(output_dir, img_name_list[i])
        cv2.imwrite(output_path, corrected_img)

    else:
        print(f"Could not correct perspective for {img_path}")



# # dir_path의 모든 이미지를 사용한 경우
# for i, img_path in enumerate(img_list):
#   corrected_img = correct_perspective(img_path)

#   if corrected_img is not None:
#     # Open CV 이미지를 Matplotlib에서 사용하기 위헤 BGR에서 RGB로 변환
#     corrected_img_rgb = cv2.cvtColor(corrected_img, cv2.COLOR_BGR2RGB)

#     # 수정된 이미지 저장
#     output_path = os.path.join(output_dir, img_name_list[i]) # 원본 이름을 유지하여 저장
#     cv2.imwrite(output_path, corrected_img)

#   else:
#     print(f"Error: Could not correct perspective for {img_path}")

# Coordinate Extraction  
**Edited 폴더에 저장된 사진 속 공을 감지하고 좌표값을 뽑아 Coordinate.csv로 저장합니다.**

In [ ]:
# 이미지 경로
dir_path = './drive/MyDrive/Colab Notebooks/LPBA_2/Edited/'

img_name_list = [f'{str(i).zfill(4)}.png' for i in range(1, 2116)]  # 2163개의 이미지면 2164 입력
img_list = [dir_path + img_name for img_name in img_name_list]



# 색상 범위 정의 (BGR 기준)
# WHITE_LOWER = np.array([160, 181, 198], dtype=np.uint8)   # 흰색의 BGR 하한값
# WHITE_UPPER = np.array([255, 255, 255], dtype=np.uint8)   # 흰색의 BGR 상한값

# YELLOW_LOWER = np.array([20, 88, 200], dtype=np.uint8)      # 노란색의 BGR 하한값
# YELLOW_UPPER = np.array([120, 170, 255], dtype=np.uint8)    # 노란색의 BGR 상한값

# BLUE_LOWER = np.array([110, 46, 0], dtype=np.uint8)      # 파란색의 BGR 하한값
# BLUE_UPPER = np.array([200, 120, 70], dtype=np.uint8)     # 파란색의 BGR 상한값

WHITE_LOWER = np.array([145, 170, 190], dtype=np.uint8)   # 흰색의 BGR 하한값
WHITE_UPPER = np.array([255, 255, 255], dtype=np.uint8)   # 흰색의 BGR 상한값

YELLOW_LOWER = np.array([5, 80, 190], dtype=np.uint8)      # 노란색의 BGR 하한값
YELLOW_UPPER = np.array([100, 200, 255], dtype=np.uint8)    # 노란색의 BGR 상한값

RED_LOWER = np.array([0, 0, 150], dtype=np.uint8)      # 빨간색의 BGR 하한값
RED_UPPER = np.array([80, 80, 255], dtype=np.uint8)     # 빨간색의 BGR 상한값



# 원 중심으로부터 반경 r 이내 픽셀 색상 판별 함수
def detect_circle_color(img, x, y, r=7):
    # 원 영역의 좌표 범위를 설정
    x_min = max(0, x - r)
    x_max = min(img.shape[1], x + r)
    y_min = max(0, y - r)
    y_max = min(img.shape[0], y + r)

    # 해당 영역의 모든 픽셀 추출
    roi = img[y_min:y_max, x_min:x_max]

    # 각 색상의 마스크 적용
    white_mask = cv2.inRange(roi, WHITE_LOWER, WHITE_UPPER)
    yellow_mask = cv2.inRange(roi, YELLOW_LOWER, YELLOW_UPPER)
    red_mask = cv2.inRange(roi, RED_LOWER, RED_UPPER)

    # 각 마스크에 해당하는 픽셀 수 계산
    white_count = np.sum(white_mask > 0)
    yellow_count = np.sum(yellow_mask > 0)
    red_count = np.sum(red_mask > 0)

    # 가장 많은 픽셀을 가진 색상 반환
    if white_count > yellow_count and white_count > red_count:
        return "White"

    elif yellow_count > red_count:
        return "Yellow"

    else:
        return "Red"



# 결과를 저장할 리스트
results = []

for img_path in img_list:
    # 이미지별로 저장할 딕셔너리 (각 색상에 맞는 x, y 좌표를 넣을 예정)
    img_results = {'x_white': np.nan, 'y_white': np.nan,
                   'x_yellow': np.nan, 'y_yellow': np.nan,
                   'x_red': np.nan, 'y_red': np.nan}

    # 이미지 파일이 존재하는지 확인
    if not os.path.exists(img_path):
        print(f"Error: Image not found: {img_path}")

        results.append(img_results)

        continue

    # 이미지 불러오기
    img = cv2.imread(img_path)

    # 이미지가 제대로 불러와졌는지 확인
    if img is None:
        print(f"Error: Failed to load image: {img_path}")

        results.append(img_results)

        continue

    ######### 배경제거 #########
    # 색 대체
    image_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 배경 마스크 범위 설정 (hsv 기준) - 파란색 당구대 보드
    mask_low = np.array([90, 100, 100])
    mask_high = np.array([130, 255, 255])

    # 배경 마스크
    mask = cv2.inRange(image_hsv, mask_low, mask_high)

    # 배경 -> 검정색 변환
    image_replace = img.copy()
    image_replace[mask > 0] = (0, 0, 0)        # 배경 검정색으로 대체
    image_replace[mask == 0] = (255, 255, 255) # 나머지는 모두 흰색 변환

    # 원 검출 가능하도록 GRAYSCALE 변환
    image_gray = cv2.cvtColor(image_replace, cv2.COLOR_BGR2GRAY)

    # 원 검출
    circles = cv2.HoughCircles(
        image_gray,
        cv2.HOUGH_GRADIENT,
        dp=1.1,            # 해상도
        minDist=12,        # 공들 간의 최소 거리
        param1=50,         # Canny 에지 검출의 상한
        param2=12,         # 검출 민감도
        minRadius=5,       # 최소 반지름
        maxRadius=10       # 최대 반지름
    )

    # 원 감지 결과 출력 및 좌표와 색상 추출
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")

        for (x, y, r) in circles:
            # print(f"Ball center: ({x}, {y}), Radius: {r}")

            # 감지된 원의 색상 확인
            color = detect_circle_color(img, x, y)
            # print(f"Detected color: {color}")

            # 원의 색상에 따라 좌표를 img_results 딕셔너리에 저장
            if color == 'White':
                img_results['x_white'] = x
                img_results['y_white'] = y

            elif color == 'Yellow':
                img_results['x_yellow'] = x
                img_results['y_yellow'] = y

            elif color == 'Red':
                img_results['x_red'] = x
                img_results['y_red'] = y

    # 결과 저장 (이미지별 데이터)
    results.append(img_results)



# 결과를 데이터프레임으로 변환
df_mid = pd.DataFrame(results)

# Error 이미지 수 확인
df_mid_null = df_mid[df_mid.isnull().any(axis=1)]
print("Error Images Count:", len(df_mid_null))



excel = pd.read_excel('./drive/MyDrive/Colab Notebooks/LPBA_3/Data.xlsx')



def create_df_last(df_mid, excel):
    data = {
        '큐볼_x': [],
        '큐볼_y': [],
        '적구_1_x': [],
        '적구_1_y': [],
        '적구_2_x': [],
        '적구_2_y': []
    }

    for i, row in excel.iterrows():
        # Ball 값에 따라 큐볼 좌표를 결정
        if row['Ball'] == 1:
            # 큐볼: 흰 공, 적구: 노란 공과 빨간 공
            큐볼_x, 큐볼_y = df_mid.loc[i, ['x_white', 'y_white']]
            적구_1_x, 적구_1_y = df_mid.loc[i, ['x_yellow', 'y_yellow']]
            적구_2_x, 적구_2_y = df_mid.loc[i, ['x_red', 'y_red']]

        elif row['Ball'] == 2:
            # 큐볼: 노란 공, 적구: 흰 공과 빨간 공
            큐볼_x, 큐볼_y = df_mid.loc[i, ['x_yellow', 'y_yellow']]
            적구_1_x, 적구_1_y = df_mid.loc[i, ['x_white', 'y_white']]
            적구_2_x, 적구_2_y = df_mid.loc[i, ['x_red', 'y_red']]

        else:
            # Ball 값이 1 또는 2가 아닌 경우 처리
            continue

        # 데이터를 리스트에 추가
        data['큐볼_x'].append(큐볼_x)
        data['큐볼_y'].append(큐볼_y)
        data['적구_1_x'].append(적구_1_x)
        data['적구_1_y'].append(적구_1_y)
        data['적구_2_x'].append(적구_2_x)
        data['적구_2_y'].append(적구_2_y)

    # 최종 데이터프레임 생성
    df_last = pd.DataFrame(data)

    return df_last



# df_last 데이터프레임 생성
df_last = create_df_last(df_mid, excel)

# Error 이미지 수 확인 2
df_last_null = df_last[df_last.isnull().any(axis=1)]
print("Error Images Count 2:", len(df_last_null))



# 좌표 정보 Export
df_last.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/Coordinate.csv')

Error Images Count: 253
Error Images Count 2: 253


In [ ]:
df_last.head(10)

,큐볼_x,큐볼_y,적구_1_x,적구_1_y,적구_2_x,적구_2_y
0,575.0,151.0,387.0,153.0,199.0,152.0
1,59.0,290.0,287.0,208.0,177.0,83.0
2,278.0,97.0,404.0,169.0,276.0,369.0
3,45.0,254.0,126.0,366.0,144.0,318.0
4,227.0,95.0,43.0,172.0,199.0,337.0
5,345.0,343.0,559.0,54.0,463.0,309.0
6,648.0,266.0,182.0,339.0,298.0,31.0
7,650.0,368.0,747.0,233.0,742.0,299.0
8,524.0,312.0,260.0,119.0,741.0,298.0
9,703.0,325.0,359.0,334.0,166.0,51.0


# Image Creation  
**Coordinate.csv에 기재된 공 별 x, y 좌표값을 이용해 이미지를 생성하고 Created 폴더에 저장합니다.**

In [ ]:
df_last = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/Coordinate.csv')
df_last.drop('Unnamed: 0', axis=1, inplace=True)
df_last.head(10)

,큐볼_x,큐볼_y,적구_1_x,적구_1_y,적구_2_x,적구_2_y
0,575.0,151.0,387.0,153.0,199.0,152.0
1,59.0,290.0,287.0,208.0,177.0,83.0
2,278.0,97.0,404.0,169.0,276.0,369.0
3,45.0,254.0,126.0,366.0,144.0,318.0
4,227.0,95.0,43.0,172.0,199.0,337.0
5,345.0,343.0,559.0,54.0,463.0,309.0
6,648.0,266.0,182.0,339.0,298.0,31.0
7,650.0,368.0,747.0,233.0,742.0,299.0
8,524.0,312.0,260.0,119.0,741.0,298.0
9,703.0,325.0,359.0,334.0,166.0,51.0


In [ ]:
co = df_last

# 쿠션 고려하여 원점 이동 ((13, 13) -> (0, 0))
co = co - 13



# 저장할 디렉토리 경로 설정
output_dir = "./drive/MyDrive/Colab Notebooks/LPBA_3/Created/"

# 디렉토리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)



# 이미지 크기 설정
img_width = 753
img_height = 376
radius = 9  # 공의 반지름



# 테두리 넘어가는 공을 조정하는 함수
def adjust_coordinates(x, y, radius, img_width, img_height):
    if x - radius < 0:
        x = radius

    elif x + radius > img_width:
        x = img_width - radius

    if y - radius < 0:
        y = radius

    elif y + radius > img_height:
        y = img_height - radius

    return x, y



# 두 공 사이의 거리를 계산하는 함수
def distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)



# 공들이 겹쳤을 경우 좌표를 보정하는 함수
def adjust_if_overlapping(x1, y1, x2, y2, radius):
    dist = distance(x1, y1, x2, y2)

    if dist < 2 * radius:  # 공들이 겹쳤을 경우
        overlap = 2 * radius - dist
        angle = math.atan2(y2 - y1, x2 - x1)

        # 두 공이 접하도록 위치를 조정 (겹침 해소)
        x2 += overlap * math.cos(angle)
        y2 += overlap * math.sin(angle)

        # 두 공이 이미지 범위를 넘지 않도록 재조정
        x2, y2 = adjust_coordinates(x2, y2, radius, img_width, img_height)

    return x2, y2



# 모든 행에 대해 처리
for index, row in co.iterrows():

    # 이미지 생성 (배경은 투명)
    img = Image.new('RGBA', (img_width, img_height), (0, 0, 0, 0))
    draw = ImageDraw.Draw(img)

    # 큐볼의 좌표 조정
    first_x, first_y = adjust_coordinates(row['큐볼_x'], row['큐볼_y'], radius, img_width, img_height)

    # 적구_1과 적구_2의 좌표 조정
    second_x, second_y = adjust_coordinates(row['적구_1_x'], row['적구_1_y'], radius, img_width, img_height)
    third_x, third_y = adjust_coordinates(row['적구_2_x'], row['적구_2_y'], radius, img_width, img_height)

    # 공끼리 겹치는지 확인하고 좌표 보정
    second_x, second_y = adjust_if_overlapping(first_x, first_y, second_x, second_y, radius)
    third_x, third_y = adjust_if_overlapping(first_x, first_y, third_x, third_y, radius)
    third_x, third_y = adjust_if_overlapping(second_x, second_y, third_x, third_y, radius)

    # 공 색상 설정: 큐볼(노란색), 적구 2개(빨간색)
    draw.ellipse((first_x - radius, first_y - radius, first_x + radius, first_y + radius), fill=(255, 255, 0, 255))  # 큐볼 (노란색)
    draw.ellipse((second_x - radius, second_y - radius, second_x + radius, second_y + radius), fill=(255, 0, 0, 255))  # 적구_1 (빨간색)
    draw.ellipse((third_x - radius, third_y - radius, third_x + radius, third_y + radius), fill=(255, 0, 0, 255))  # 적구_2 (빨간색)

    # 이미지 파일로 저장 (index를 사용하여 파일명을 구분)
    img.save(os.path.join(output_dir, f'{str(index + 4585).zfill(4)}.png'))  # 파일명을 4585부터 시작